In [ ]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
import matplotlib.animation as animation
import numpy as np

#############################
# 1) Define Your Custom List of Players
#############################
specific_players = [
    "FiveStarGeneral", "Octane", "Xenophon", "Platinum", 
    "Jacoþ thε Restle§°ⁿ³", "MarkusBM", 
    "Ursus", "7ate9", "Crimson King",
    "nickpugs3", "Ralph", "Orcus", "bast", "jache", "Jack Norris"
]
print("Custom player list:", specific_players)

#############################
# 2) Fetch Player IDs & Rating History from December 1, 2024 till today
#############################

# your path
db_path = "/Users/.../mtl.db"
conn = sqlite3.connect(db_path)

# Build placeholders for specific players
placeholders = ",".join(["?"] * len(specific_players))
query_player_ids = f"""
SELECT id, name
FROM player
WHERE name IN ({placeholders})
"""
df_player_ids = pd.read_sql_query(query_player_ids, conn, params=specific_players)
if df_player_ids.empty:
    print("No matching players found for the specified names.")
    conn.close()
    raise SystemExit

player_ids = df_player_ids["id"].tolist()
placeholders_ids = ",".join(["?"] * len(player_ids))
query_history = f"""
SELECT recorded_date, player_id, rating
FROM player_history
WHERE player_id IN ({placeholders_ids})
  AND recorded_date >= '2024-12-01'
ORDER BY recorded_date
"""
df_history = pd.read_sql_query(query_history, conn, params=player_ids)
conn.close()

if df_history.empty:
    print("No rating history found for these players since 2024-12-01.")
    raise SystemExit

#############################
# 3) Merge Data & Convert Dates
#############################
df_merged = pd.merge(df_history, df_player_ids, left_on="player_id", right_on="id", how="inner")
df_merged["recorded_date"] = pd.to_datetime(df_merged["recorded_date"])
df_merged.rename(columns={"name": "player_name"}, inplace=True)
df_merged = df_merged.sort_values(by="recorded_date")

#############################
# 4) Determine Order by Max Rating
#############################
max_ratings = df_merged.groupby("player_name")["rating"].max().sort_values(ascending=False)
ordered_players = max_ratings.index.tolist()
print("Players sorted by max rating:\n", max_ratings)

#############################
# 5) Prepare Data for Animation
#############################
# Get unique dates (each frame) and store each player's dates and ratings.
unique_dates = df_merged["recorded_date"].sort_values().unique()
print(f"Animating over {len(unique_dates)} time steps.")

player_data = {}
for player in ordered_players:
    df_p = df_merged[df_merged["player_name"] == player]
    player_data[player] = {
        "dates": df_p["recorded_date"].values,
        "ratings": df_p["rating"].values
    }

#############################
# 6) Create the Animation with Continuous Annotations
#############################
fig, ax = plt.subplots(figsize=(12, 6))
ax.set_xlim(df_merged["recorded_date"].min(), df_merged["recorded_date"].max())
ax.set_ylim(df_merged["rating"].min() - 50, df_merged["rating"].max() + 50)

# Set x-axis ticks every 7 days
ax.xaxis.set_major_locator(mdates.DayLocator(interval=7))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))

# Initialize line objects and annotation text objects for each player.
palette = sns.color_palette("tab10", n_colors=len(ordered_players))
lines = {}
annotations = {}
for i, player in enumerate(ordered_players):
    linestyle = "-" if i % 2 == 0 else "--"
    color = palette[i]
    line, = ax.plot([], [], label=player, color=color, linestyle=linestyle)
    lines[player] = line
    # Create annotation for each player; it will be updated each frame.
    annotations[player] = ax.text(0, 0, player, fontsize=8, color=color, weight="bold")

# Legend ordered by max rating
handles, labels = ax.get_legend_handles_labels()
ordered_handles = [handles[labels.index(p)] for p in ordered_players if p in labels]
ax.legend(ordered_handles, ordered_players, title="Player", bbox_to_anchor=(1.05, 1), loc="upper left")

# Horizontal dashed reference lines 
for threshold in [1900, 2000, 2100]:
    ax.axhline(y=threshold, color="gray", linestyle="--", linewidth=0.7, alpha=0.5)

def update(frame):
    current_date = unique_dates[frame]
    ax.set_title(f"Rating as of {pd.to_datetime(current_date).date()}")
    for player in ordered_players:
        dates = player_data[player]["dates"]
        ratings = player_data[player]["ratings"]
        mask = dates <= current_date
        x = dates[mask]
        y = ratings[mask]
        lines[player].set_data(x, y)
        # Continuously update the annotation position so it is always visible.
        if len(x) > 0:
            x_last = x[-1]
            y_last = y[-1]
            annotations[player].set_position((x_last, y_last))
            annotations[player].set_text(player)
    return list(lines.values()) + list(annotations.values())

# Use a slower interval (e.g., 500ms per frame)
ani = animation.FuncAnimation(fig, update, frames=len(unique_dates), interval=500, blit=False)

plt.tight_layout()
plt.show()

#############################
# 7) Save the Animation as GIF (open GIF in chrome for example)
#############################
from matplotlib.animation import PillowWriter
writer = PillowWriter(fps=5)  # Slow animation pace: 5 frames per second
ani.save("rating_animation.gif", writer=writer)
